In [ ]:
import orbipy as op, numpy as np, pandas as pd
import matplotlib.pyplot as plt, csv

model = op.crtbp3_model('Earth-Moon (default)')
plot_creator = op.plotter.from_model(model, length_units='Mm')
scaler = op.scaler.from_model(model)

In [ ]:
disturbance = 1e-6
model.integrator.set_params(max_step=scaler(1, 'h-nd'))

# Necessary Methods — Definition

In [ ]:
def determine_state(model, time_sequence, state, mode, disturbance=1e-6, debug=False): 
    result = np.empty((len(time_sequence), 6))
    sign = 1 if mode == 'unstable' else -1
    
    for index, time in enumerate(time_sequence):
        arc_data = model.prop(state, 0., time)
        inter_state = arc_data.iloc[-1].values[1: ]
        
        # += disturbance in case of transition to right, else -= disturbance
        inter_state[3] -= disturbance
        surface = op.eventX(1 - model.mu, terminal=True, count=1)
        
        detector = op.event_detector(model, [surface])
        trajectory_data, event_data = detector.prop(inter_state, 0., sign * 10. * np.pi, last_state='last')

        result[index] = event_data.iloc[-1].values[4: ] 
    return result

In [ ]:
def check_segment_intersection(a_from, a_to, b_from, b_to):
    p = a_from; r = a_to - a_from
    q = b_from; u = b_to - b_from
    
    rxu = np.cross(r, u)
    qpr = np.cross(q - p, r)
    
    if rxu == 0: # parallel
        return False
    
    t = np.cross(q - p, u) / rxu
    l = qpr / rxu
    
    if 0. <= t <= 1. and 0. <= l <= 1.: # intersecting
        return True
    else: # not intersecting
        return False

In [ ]:
def precise_intersection_search(unstable_parameters, stable_parameters, epsilon=1e-8, number=40, verbose=False):
    '''
    unstable_parameters: tuple of unstable state and period
    stable_parameters: tuple of stable state and period
    
    return: unstable time parameters, stable time parameters and velocity corrections
    '''
    
    unstable_state, unstable_period = unstable_parameters
    stable_state, stable_period = stable_parameters
    
    unstable_times, stable_times = [(0., unstable_period)], [(0., stable_period)]
    velocity_corrections = []
    tolerance = 1.
    intersection_number = 1
    
    while tolerance > epsilon:
        
        # Enter and out time parameters are going to be updated (precised)
        enter_updates, out_updates, velocity_updates = [], [], []
        current_tolerance = 0.
        
        for pair_index in range(intersection_number):
            
            # Unstable time interval partition to 'number' parts and corresponding positions calculation
            unstable_left, unstable_right = unstable_times[pair_index]
            unstable_time_sequence = np.linspace(unstable_left, unstable_right, number)
            
            enter_state = determine_state(model, unstable_time_sequence, unstable_state, 'unstable')
            enter_position, enter_velocities = enter_state[:, 1: 3], enter_state[:, 3: ]
            
            # Stable time interval partition to 'number' parts and corresponding positions calculation
            stable_left, stable_right = stable_times[pair_index]
            stable_time_sequence = np.linspace(stable_left, stable_right, number)

            out_state = determine_state(model, stable_time_sequence, stable_state, 'stable')
            out_position, out_velocities = out_state[:, 1: 3], out_state[:, 3: ]

            # Time parts (thereby positions) enumeration
            for enter_index in range(len(enter_position) - 1):
                for out_index in range(len(out_position) - 1):

                    # Consequent positions are percieved as segments
                    enter_left, enter_right = enter_position[enter_index], enter_position[enter_index + 1]
                    out_left, out_right = out_position[out_index], out_position[out_index + 1]

                    # Check intersection, update time parameters and current tolerance
                    if check_segment_intersection(enter_left, enter_right, out_left, out_right):

                        enter_updates.append((unstable_time_sequence[enter_index], 
                                              unstable_time_sequence[enter_index + 1]))
                        out_updates.append((stable_time_sequence[out_index], 
                                            stable_time_sequence[out_index + 1]))

                        current_tolerance = max(np.linalg.norm(enter_right - enter_left), current_tolerance)
                        velocity_updates.append(out_velocities[out_index + 1] - enter_velocities[enter_index + 1])

        # Update result with precised time parameters and velocity corrections
        unstable_times, stable_times = enter_updates, out_updates
        velocity_corrections = velocity_updates
        intersection_number = len(unstable_times)
        
        # Change current tolerance
        tolerance = current_tolerance
        
        # Two parts for time interval — sufficient
        number = 3
        if verbose:
            print('Tolerance:', tolerance, end='\n')
            
    return [pair[-1] for pair in unstable_times], [pair[-1] for pair in stable_times], velocity_corrections

# Data Preparation

In [ ]:
path_area_L1, path_area_L2 = '', ''
area_L1, area_L2 = pd.read_csv(path_area_L1), pd.read_csv(path_area_L2)

In [ ]:
state_number = 0
unstable_state = area_L2.iloc[state_number].values

stable_period = unstable_period = 1.1 * np.pi
boundaries = (0, len(area_L1.index))

# Intersection Search

In [ ]:
report_name = 'Report.csv'

In [ ]:
verbose = True
transform = lambda correction: scaler(np.linalg.norm(correction), 'nd/nd-m/s')

with open(report_name, 'a+') as report:
    reporter = csv.writer(report, delimiter=',', lineterminator='\n')
    
    for index in range(*boundaries):
        if verbose:
            print(index)

        # Set current stable state
        stable_state = area_L1.iloc[index].values

        # Investigate intersections
        unstable_parameters = (unstable_state, unstable_period)
        stable_parameters = (stable_state, stable_period)
        result = precise_intersection_search(unstable_parameters, stable_parameters, verbose=True)

        # Return velocity norms, search the best
        unstable_times, stable_times, velocity_corrections = result
        velocity_norms = list(map(transform, velocity_corrections))
        
        if not len(velocity_norms):
            print('Intersection not found...')
            continue
        
        if verbose:
            print('Intersection found!')
        
        best = np.argmin(velocity_norms)
        data = [*unstable_state, unstable_times[best], *stable_state, stable_times[best], *velocity_corrections[best]]
        
        reporter.writerow(data)
        if verbose:
            print('Data saved!')